In [ ]:
!pip install alpaca-py --quiet
!pip install yfinance

In [ ]:
import json
from datetime import datetime, timedelta
import time
from zoneinfo import ZoneInfo
import os
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import spacy

import alpaca
from alpaca.data.live.stock import *
from alpaca.data.historical.stock import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.trading.client import *
from alpaca.trading.stream import *
from alpaca.trading.requests import *
from alpaca.trading.enums import *
from alpaca.common.exceptions import APIError

In [ ]:
api_key = "PKVTX6VPYFXDRI4WDE9Q"
secret_key = "MV5dh2Ddom0GHP3mNNDVcs5YuKJqUbBbzgiyhlUK"


paper = True
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [ ]:
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [ ]:
companies = ['SPY', 'TSLA', 'AAPL', 'AMZN', 'META', 'MSFT']

In [ ]:
for symbol in companies:
  now = datetime.now(ZoneInfo("Europe/Moscow"))
  req = StockBarsRequest(
      symbol_or_symbols = [symbol],
      timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Day),
      start = '2022-01-03'
  )
  data = stock_historical_data_client.get_stock_bars(req).df
  data.reset_index(inplace=True)
  data = data[["timestamp",'open',"close",'volume']]
  data['timestamp'] = data['timestamp'].dt.tz_localize(None)
  data['timestamp'] = data['timestamp'].dt.date
  data.to_csv("/content/{symbol}.csv".format(symbol=symbol))

In [ ]:
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKVTX6VPYFXDRI4WDE9Q",
    "APCA-API-SECRET-KEY": "MV5dh2Ddom0GHP3mNNDVcs5YuKJqUbBbzgiyhlUK"
}

In [ ]:
url = "https://data.alpaca.markets/v1beta1/news?start=2022-01-03&sort=asc&limit=50"
response = requests.get(url, headers=headers)
result = response.json()['news']
t_sleep = 0.2
latest_page = ''
for i in range(5000):
  latest_page = response.json()['next_page_token']
  url = "https://data.alpaca.markets/v1beta1/news?start=2022-01-03&sort=asc&limit=50&page_token={next_page}".format(next_page=response.json()['next_page_token'])
  response = requests.get(url, headers=headers)
  result += response.json()['news']
  time.sleep(t_sleep)

In [ ]:
def get_ticker(company_name):

    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, 'lang': 'en-US'}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']

    return company_code

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
company_ticker = []
for index, row in df1.iterrows():
  temp = []
  text = row['headline']
  doc = nlp(text)
  for ent in doc.ents:
    if ent.label_ =='ORG':
      try:
        temp.append(get_ticker(ent.text))
      except:
        continue

  if len(temp)!==0:
    company_ticker.append(temp)

In [8]:
from google.colab import data_table

data_table.enable_dataframe_formatter()

In [71]:
df = pd.read_csv('/content/news_from_2022-01-03T02_00_37Z_till_2024-05-22T05_21_42Z.csv')

In [64]:
df

,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at
0,0.0,2022-01-03 02:00:37+00:00,"Bitcoin, Ethereum, Dogecoin Santa Rally Held B...",NaN,Ethereum’s (CRYPTO: ETH) 24-hour gains outstri...,"['$BTC', '$DOGE', '$ETH']",2022-01-03 02:00:38+00:00
1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,['AAPL'],2022-01-03 04:04:21+00:00
2,2.0,2022-01-03 02:22:33+00:00,"Robinhood, DraftKings — Cathie Wood's Last Tra...",NaN,Cathie Wood’s Ark Investment Management on Fri...,"['$BTC', '$DOGE', '$ETH', 'ARKF', 'ARKK', 'ARK...",2022-01-03 04:19:23+00:00
3,3.0,2022-01-03 05:19:17+00:00,"This Knockoff Coin is Up 164% Today, Outperfor...",NaN,Some of the Dogecoin (CRYPTO: DOGE) and Shiba ...,"['$DOGE', '$SHIB']",2022-01-03 05:19:17+00:00
4,4.0,2022-01-03 05:24:12+00:00,"Alibaba, JD Fall As Evergrande Suspends Hong K...",NaN,Shares of Alibaba Group Holding Limited (NYSE:...,"['BABA', 'BIDU', 'EGRNY', 'JD', 'LI', 'NIO', '...",2022-01-03 05:24:13+00:00
...,...,...,...,...,...,...,...
614338,614336.0,2024-05-22 04:55:11+00:00,"Reported Earlier, Japan Trade Balance For Apri...",benzinga,NaN,"['BBJP', 'DXJ', 'EWJ']",2024-05-22 04:55:11+00:00
614339,614337.0,2024-05-22 05:11:12+00:00,Cool Co Q1 Sales $88.125 Miss $86.500M Estimat...,benzinga,NaN,['CLCO'],2024-05-22 05:11:13+00:00
614340,614338.0,2024-05-22 05:12:09+00:00,Donald Trump's Foreign Policy Advisor Calls Fo...,benzinga,"Robert O&#39;Brien, a key foreign policy advis...",[],2024-05-22 05:12:10+00:00
614341,614339.0,2024-05-22 05:13:51+00:00,Sandoz Obtains European Commission Approval Fo...,benzinga,NaN,['SDZNY'],2024-05-22 05:13:52+00:00


In [72]:
df.dropna()
df

,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at
0,0.0,2022-01-03 02:00:37+00:00,"Bitcoin, Ethereum, Dogecoin Santa Rally Held B...",NaN,Ethereum’s (CRYPTO: ETH) 24-hour gains outstri...,"['$BTC', '$DOGE', '$ETH']",2022-01-03 02:00:38+00:00
1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,['AAPL'],2022-01-03 04:04:21+00:00
2,2.0,2022-01-03 02:22:33+00:00,"Robinhood, DraftKings — Cathie Wood's Last Tra...",NaN,Cathie Wood’s Ark Investment Management on Fri...,"['$BTC', '$DOGE', '$ETH', 'ARKF', 'ARKK', 'ARK...",2022-01-03 04:19:23+00:00
3,3.0,2022-01-03 05:19:17+00:00,"This Knockoff Coin is Up 164% Today, Outperfor...",NaN,Some of the Dogecoin (CRYPTO: DOGE) and Shiba ...,"['$DOGE', '$SHIB']",2022-01-03 05:19:17+00:00
4,4.0,2022-01-03 05:24:12+00:00,"Alibaba, JD Fall As Evergrande Suspends Hong K...",NaN,Shares of Alibaba Group Holding Limited (NYSE:...,"['BABA', 'BIDU', 'EGRNY', 'JD', 'LI', 'NIO', '...",2022-01-03 05:24:13+00:00
...,...,...,...,...,...,...,...
614338,614336.0,2024-05-22 04:55:11+00:00,"Reported Earlier, Japan Trade Balance For Apri...",benzinga,NaN,"['BBJP', 'DXJ', 'EWJ']",2024-05-22 04:55:11+00:00
614339,614337.0,2024-05-22 05:11:12+00:00,Cool Co Q1 Sales $88.125 Miss $86.500M Estimat...,benzinga,NaN,['CLCO'],2024-05-22 05:11:13+00:00
614340,614338.0,2024-05-22 05:12:09+00:00,Donald Trump's Foreign Policy Advisor Calls Fo...,benzinga,"Robert O&#39;Brien, a key foreign policy advis...",[],2024-05-22 05:12:10+00:00
614341,614339.0,2024-05-22 05:13:51+00:00,Sandoz Obtains European Commission Approval Fo...,benzinga,NaN,['SDZNY'],2024-05-22 05:13:52+00:00


In [73]:
df['symbols'] = df['symbols'].str.strip('[]').str.split(',')

In [38]:
df['symbols']

0                               ['$BTC',  '$DOGE',  '$ETH']
1                                                  ['AAPL']
2         ['$BTC',  '$DOGE',  '$ETH',  'ARKF',  'ARKK', ...
3                                       ['$DOGE',  '$SHIB']
4         ['BABA',  'BIDU',  'EGRNY',  'JD',  'LI',  'NI...
                                ...                        
614338                             ['BBJP',  'DXJ',  'EWJ']
614339                                             ['CLCO']
614340                                                   []
614341                                            ['SDZNY']
614342                                                   []
Name: symbols, Length: 614343, dtype: object

In [74]:
mask = df['symbols'].str.len()
df = df[mask==1]

In [75]:
df

,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at
1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,['AAPL'],2022-01-03 04:04:21+00:00
6,6.0,2022-01-03 07:46:58+00:00,Dogecoin Knockoff Project Launches Meme-Coin L...,NaN,"DogeBonk (DOBO), a Dogecoin (CRYPTO: DOGE) kno...",['$DOGE'],2022-01-03 07:46:59+00:00
7,7.0,2022-01-03 08:13:27+00:00,"Robust iPhone Demand, Services Will Chart Appl...",NaN,Apple Inc (NASDAQ: AAPL) analyst Dan Ives says...,['AAPL'],2022-01-03 08:13:28+00:00
15,15.0,2022-01-03 09:57:33+00:00,Dogecoin Trades Flat To Start 2022: Is 'Pumpin...,NaN,Dogecoin (CRYPTO: DOGE) traded mostly unchange...,['$DOGE'],2022-01-03 09:57:34+00:00
16,16.0,2022-01-03 10:04:18+00:00,Economic Data Scheduled For Monday,NaN,\n\tThe manufacturing PMI for December is sche...,[],2022-01-03 10:04:18+00:00
...,...,...,...,...,...,...,...
614335,614333.0,2024-05-22 04:27:33+00:00,Apple Challenges DOJ's Antitrust Case Saying T...,benzinga,Lawyers representing Apple have sent a letter ...,['AAPL'],2024-05-22 04:27:34+00:00
614339,614337.0,2024-05-22 05:11:12+00:00,Cool Co Q1 Sales $88.125 Miss $86.500M Estimat...,benzinga,NaN,['CLCO'],2024-05-22 05:11:13+00:00
614340,614338.0,2024-05-22 05:12:09+00:00,Donald Trump's Foreign Policy Advisor Calls Fo...,benzinga,"Robert O&#39;Brien, a key foreign policy advis...",[],2024-05-22 05:12:10+00:00
614341,614339.0,2024-05-22 05:13:51+00:00,Sandoz Obtains European Commission Approval Fo...,benzinga,NaN,['SDZNY'],2024-05-22 05:13:52+00:00


In [86]:
mask1 = df['symbols'].str[0]!=''
mask2 = df['symbols'].str[0] == "'SPY'"
mask3 = df['symbols'].str[0] == "'TSLA'"
mask4 = df['symbols'].str[0] == "'AAPL'"
mask5 = df['symbols'].str[0] == "'AMZN'"
mask6 = df['symbols'].str[0] == "'META'"
mask7 = df['symbols'].str[0] == "'MSFT'"
df = df[mask1 & (mask2 | mask3 | mask4 | mask5 | mask6 | mask7)]

In [87]:
df

,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at
1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,['AAPL'],2022-01-03 04:04:21+00:00
7,7.0,2022-01-03 08:13:27+00:00,"Robust iPhone Demand, Services Will Chart Appl...",NaN,Apple Inc (NASDAQ: AAPL) analyst Dan Ives says...,['AAPL'],2022-01-03 08:13:28+00:00
17,17.0,2022-01-03 10:05:28+00:00,Tesla Shares Surge 7% Premarket After Report o...,NaN,NaN,['TSLA'],2022-01-03 10:05:28+00:00
62,62.0,2022-01-03 11:47:02+00:00,Read Why JPMorgan Bumped Up Tesla Price Target...,NaN,JPMorgan analyst Ryan Brinkman raised the pric...,['TSLA'],2022-01-03 11:47:03+00:00
185,185.0,2022-01-03 13:53:32+00:00,Why Tesla Shares Are Trading Higher Today,NaN,Tesla Inc (NASDAQ: TSLA) is trading higher Mon...,['TSLA'],2022-01-03 13:53:32+00:00
...,...,...,...,...,...,...,...
614319,614317.0,2024-05-22 00:50:38+00:00,Elon Musk Has 'Fond Memories' From Red Lobster...,benzinga,,['TSLA'],2024-05-22 00:50:38+00:00
614325,614323.0,2024-05-22 02:13:23+00:00,Microsoft Not In Control Of Its AI Destiny Lik...,benzinga,Gene Munster of Deepwater Asset Management cri...,['MSFT'],2024-05-22 02:13:24+00:00
614330,614328.0,2024-05-22 03:01:34+00:00,Microsoft CEO Satya Nadella Says Stop Treating...,benzinga,"Satya Nadella, CEO of Microsoft, has stated th...",['MSFT'],2024-05-22 03:01:34+00:00
614332,614330.0,2024-05-21 20:13:53+00:00,Scarlett Johansson's Clash With OpenAI Over Ch...,benzinga,Johansson declined OpenAI&#39;s request to use...,['MSFT'],2024-05-22 03:40:36+00:00


In [88]:
df = df.dropna(subset=['headline'])

In [89]:
df

,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at
1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,['AAPL'],2022-01-03 04:04:21+00:00
7,7.0,2022-01-03 08:13:27+00:00,"Robust iPhone Demand, Services Will Chart Appl...",NaN,Apple Inc (NASDAQ: AAPL) analyst Dan Ives says...,['AAPL'],2022-01-03 08:13:28+00:00
17,17.0,2022-01-03 10:05:28+00:00,Tesla Shares Surge 7% Premarket After Report o...,NaN,NaN,['TSLA'],2022-01-03 10:05:28+00:00
62,62.0,2022-01-03 11:47:02+00:00,Read Why JPMorgan Bumped Up Tesla Price Target...,NaN,JPMorgan analyst Ryan Brinkman raised the pric...,['TSLA'],2022-01-03 11:47:03+00:00
185,185.0,2022-01-03 13:53:32+00:00,Why Tesla Shares Are Trading Higher Today,NaN,Tesla Inc (NASDAQ: TSLA) is trading higher Mon...,['TSLA'],2022-01-03 13:53:32+00:00
...,...,...,...,...,...,...,...
614319,614317.0,2024-05-22 00:50:38+00:00,Elon Musk Has 'Fond Memories' From Red Lobster...,benzinga,,['TSLA'],2024-05-22 00:50:38+00:00
614325,614323.0,2024-05-22 02:13:23+00:00,Microsoft Not In Control Of Its AI Destiny Lik...,benzinga,Gene Munster of Deepwater Asset Management cri...,['MSFT'],2024-05-22 02:13:24+00:00
614330,614328.0,2024-05-22 03:01:34+00:00,Microsoft CEO Satya Nadella Says Stop Treating...,benzinga,"Satya Nadella, CEO of Microsoft, has stated th...",['MSFT'],2024-05-22 03:01:34+00:00
614332,614330.0,2024-05-21 20:13:53+00:00,Scarlett Johansson's Clash With OpenAI Over Ch...,benzinga,Johansson declined OpenAI&#39;s request to use...,['MSFT'],2024-05-22 03:40:36+00:00


In [90]:
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [91]:
headlines = df['headline'].to_list()

In [92]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [93]:
sent = []
tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
for line in headlines:
    with torch.no_grad():
        input_sequence = tokenizer(line, return_tensors="pt", **tokenizer_kwargs)
        logits = model(**input_sequence).logits
        scores = {
        s: p
        for s, p in zip(
            model.config.id2label.values(),
            scipy.special.softmax(logits.numpy().squeeze()),
        )
    }
    sent.append(scores)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614343 entries, 0 to 614342
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  614341 non-null  float64
 1   created_at  614343 non-null  object 
 2   headline    614343 non-null  object 
 3   source      613359 non-null  object 
 4   summary     261565 non-null  object 
 5   symbols     614339 non-null  object 
 6   updated_at  614339 non-null  object 
dtypes: float64(1), object(6)
memory usage: 32.8+ MB


In [99]:
df = df.reset_index()

In [100]:
df

,index,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at
0,1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,['AAPL'],2022-01-03 04:04:21+00:00
1,7,7.0,2022-01-03 08:13:27+00:00,"Robust iPhone Demand, Services Will Chart Appl...",NaN,Apple Inc (NASDAQ: AAPL) analyst Dan Ives says...,['AAPL'],2022-01-03 08:13:28+00:00
2,17,17.0,2022-01-03 10:05:28+00:00,Tesla Shares Surge 7% Premarket After Report o...,NaN,NaN,['TSLA'],2022-01-03 10:05:28+00:00
3,62,62.0,2022-01-03 11:47:02+00:00,Read Why JPMorgan Bumped Up Tesla Price Target...,NaN,JPMorgan analyst Ryan Brinkman raised the pric...,['TSLA'],2022-01-03 11:47:03+00:00
4,185,185.0,2022-01-03 13:53:32+00:00,Why Tesla Shares Are Trading Higher Today,NaN,Tesla Inc (NASDAQ: TSLA) is trading higher Mon...,['TSLA'],2022-01-03 13:53:32+00:00
...,...,...,...,...,...,...,...,...
23241,614319,614317.0,2024-05-22 00:50:38+00:00,Elon Musk Has 'Fond Memories' From Red Lobster...,benzinga,,['TSLA'],2024-05-22 00:50:38+00:00
23242,614325,614323.0,2024-05-22 02:13:23+00:00,Microsoft Not In Control Of Its AI Destiny Lik...,benzinga,Gene Munster of Deepwater Asset Management cri...,['MSFT'],2024-05-22 02:13:24+00:00
23243,614330,614328.0,2024-05-22 03:01:34+00:00,Microsoft CEO Satya Nadella Says Stop Treating...,benzinga,"Satya Nadella, CEO of Microsoft, has stated th...",['MSFT'],2024-05-22 03:01:34+00:00
23244,614332,614330.0,2024-05-21 20:13:53+00:00,Scarlett Johansson's Clash With OpenAI Over Ch...,benzinga,Johansson declined OpenAI&#39;s request to use...,['MSFT'],2024-05-22 03:40:36+00:00


In [94]:
sentiment = pd.DataFrame(sent)
sentiment

,positive,negative,neutral
0,0.732290,0.009231,0.258479
1,0.855883,0.010463,0.133654
2,0.865843,0.073437,0.060720
3,0.755008,0.048934,0.196058
4,0.265800,0.017778,0.716422
...,...,...,...
23241,0.046524,0.349830,0.603646
23242,0.042049,0.021720,0.936230
23243,0.073536,0.086786,0.839678
23244,0.034186,0.568480,0.397333


In [102]:
res = pd.concat([df, sentiment], axis=1)
res

,index,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at,positive,negative,neutral
0,1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,['AAPL'],2022-01-03 04:04:21+00:00,0.732290,0.009231,0.258479
1,7,7.0,2022-01-03 08:13:27+00:00,"Robust iPhone Demand, Services Will Chart Appl...",NaN,Apple Inc (NASDAQ: AAPL) analyst Dan Ives says...,['AAPL'],2022-01-03 08:13:28+00:00,0.855883,0.010463,0.133654
2,17,17.0,2022-01-03 10:05:28+00:00,Tesla Shares Surge 7% Premarket After Report o...,NaN,NaN,['TSLA'],2022-01-03 10:05:28+00:00,0.865843,0.073437,0.060720
3,62,62.0,2022-01-03 11:47:02+00:00,Read Why JPMorgan Bumped Up Tesla Price Target...,NaN,JPMorgan analyst Ryan Brinkman raised the pric...,['TSLA'],2022-01-03 11:47:03+00:00,0.755008,0.048934,0.196058
4,185,185.0,2022-01-03 13:53:32+00:00,Why Tesla Shares Are Trading Higher Today,NaN,Tesla Inc (NASDAQ: TSLA) is trading higher Mon...,['TSLA'],2022-01-03 13:53:32+00:00,0.265800,0.017778,0.716422
...,...,...,...,...,...,...,...,...,...,...,...
23241,614319,614317.0,2024-05-22 00:50:38+00:00,Elon Musk Has 'Fond Memories' From Red Lobster...,benzinga,,['TSLA'],2024-05-22 00:50:38+00:00,0.046524,0.349830,0.603646
23242,614325,614323.0,2024-05-22 02:13:23+00:00,Microsoft Not In Control Of Its AI Destiny Lik...,benzinga,Gene Munster of Deepwater Asset Management cri...,['MSFT'],2024-05-22 02:13:24+00:00,0.042049,0.021720,0.936230
23243,614330,614328.0,2024-05-22 03:01:34+00:00,Microsoft CEO Satya Nadella Says Stop Treating...,benzinga,"Satya Nadella, CEO of Microsoft, has stated th...",['MSFT'],2024-05-22 03:01:34+00:00,0.073536,0.086786,0.839678
23244,614332,614330.0,2024-05-21 20:13:53+00:00,Scarlett Johansson's Clash With OpenAI Over Ch...,benzinga,Johansson declined OpenAI&#39;s request to use...,['MSFT'],2024-05-22 03:40:36+00:00,0.034186,0.568480,0.397333


In [103]:
res.to_csv('sentiment.csv')

In [117]:
res = pd.read_csv('/content/sentiment.csv')

In [118]:
res

,Unnamed: 0.1,index,Unnamed: 0,created_at,headline,source,summary,symbols,updated_at,positive,negative,neutral
0,0,1,1.0,2022-01-03 04:04:21+00:00,What A 'Strong' Apple 2022 Product Line Up Cou...,NaN,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,"[""'AAPL'""]",2022-01-03 04:04:21+00:00,0.732290,0.009231,0.258479
1,1,7,7.0,2022-01-03 08:13:27+00:00,"Robust iPhone Demand, Services Will Chart Appl...",NaN,Apple Inc (NASDAQ: AAPL) analyst Dan Ives says...,"[""'AAPL'""]",2022-01-03 08:13:28+00:00,0.855882,0.010463,0.133654
2,2,17,17.0,2022-01-03 10:05:28+00:00,Tesla Shares Surge 7% Premarket After Report o...,NaN,NaN,"[""'TSLA'""]",2022-01-03 10:05:28+00:00,0.865843,0.073437,0.060720
3,3,62,62.0,2022-01-03 11:47:02+00:00,Read Why JPMorgan Bumped Up Tesla Price Target...,NaN,JPMorgan analyst Ryan Brinkman raised the pric...,"[""'TSLA'""]",2022-01-03 11:47:03+00:00,0.755008,0.048934,0.196058
4,4,185,185.0,2022-01-03 13:53:32+00:00,Why Tesla Shares Are Trading Higher Today,NaN,Tesla Inc (NASDAQ: TSLA) is trading higher Mon...,"[""'TSLA'""]",2022-01-03 13:53:32+00:00,0.265800,0.017778,0.716422
...,...,...,...,...,...,...,...,...,...,...,...,...
23241,23241,614319,614317.0,2024-05-22 00:50:38+00:00,Elon Musk Has 'Fond Memories' From Red Lobster...,benzinga,,"[""'TSLA'""]",2024-05-22 00:50:38+00:00,0.046524,0.349830,0.603646
23242,23242,614325,614323.0,2024-05-22 02:13:23+00:00,Microsoft Not In Control Of Its AI Destiny Lik...,benzinga,Gene Munster of Deepwater Asset Management cri...,"[""'MSFT'""]",2024-05-22 02:13:24+00:00,0.042049,0.021720,0.936230
23243,23243,614330,614328.0,2024-05-22 03:01:34+00:00,Microsoft CEO Satya Nadella Says Stop Treating...,benzinga,"Satya Nadella, CEO of Microsoft, has stated th...","[""'MSFT'""]",2024-05-22 03:01:34+00:00,0.073536,0.086786,0.839678
23244,23244,614332,614330.0,2024-05-21 20:13:53+00:00,Scarlett Johansson's Clash With OpenAI Over Ch...,benzinga,Johansson declined OpenAI&#39;s request to use...,"[""'MSFT'""]",2024-05-22 03:40:36+00:00,0.034186,0.568481,0.397333


In [119]:
res = res.drop(columns=res.columns[[0,1,2, 5,8]])

In [121]:
res['created_at'] = res['created_at'].str.slice(0, 10)

In [122]:
res['sentiment_score'] = res['positive'] - res['negative']

In [123]:
res

,created_at,headline,summary,symbols,positive,negative,neutral,sentiment_score
0,2022-01-03,What A 'Strong' Apple 2022 Product Line Up Cou...,Apple Inc (NASDAQ: AAPL) had a great 2021 reve...,"[""'AAPL'""]",0.732290,0.009231,0.258479,0.723059
1,2022-01-03,"Robust iPhone Demand, Services Will Chart Appl...",Apple Inc (NASDAQ: AAPL) analyst Dan Ives says...,"[""'AAPL'""]",0.855882,0.010463,0.133654,0.845419
2,2022-01-03,Tesla Shares Surge 7% Premarket After Report o...,NaN,"[""'TSLA'""]",0.865843,0.073437,0.060720,0.792407
3,2022-01-03,Read Why JPMorgan Bumped Up Tesla Price Target...,JPMorgan analyst Ryan Brinkman raised the pric...,"[""'TSLA'""]",0.755008,0.048934,0.196058,0.706074
4,2022-01-03,Why Tesla Shares Are Trading Higher Today,Tesla Inc (NASDAQ: TSLA) is trading higher Mon...,"[""'TSLA'""]",0.265800,0.017778,0.716422,0.248021
...,...,...,...,...,...,...,...,...
23241,2024-05-22,Elon Musk Has 'Fond Memories' From Red Lobster...,,"[""'TSLA'""]",0.046524,0.349830,0.603646,-0.303306
23242,2024-05-22,Microsoft Not In Control Of Its AI Destiny Lik...,Gene Munster of Deepwater Asset Management cri...,"[""'MSFT'""]",0.042049,0.021720,0.936230,0.020329
23243,2024-05-22,Microsoft CEO Satya Nadella Says Stop Treating...,"Satya Nadella, CEO of Microsoft, has stated th...","[""'MSFT'""]",0.073536,0.086786,0.839678,-0.013250
23244,2024-05-21,Scarlett Johansson's Clash With OpenAI Over Ch...,Johansson declined OpenAI&#39;s request to use...,"[""'MSFT'""]",0.034186,0.568481,0.397333,-0.534294


In [127]:
aapl_sent = pd.read_csv('/content/aapl_sentiment.csv')

In [136]:
aapl_sent

,Unnamed: 0,timestamp,open,close,volume,sent,difference,sign_match
0,0,2022-01-03,177.830,182.01,112486091.0,0.486977,4.180,True
1,1,2022-01-04,182.640,179.70,106090378.0,0.186496,-2.940,False
2,2,2022-01-05,179.610,174.92,95142198.0,-0.051516,-4.690,True
3,3,2022-01-06,172.700,172.00,103899632.0,-0.408408,-0.700,True
4,4,2022-01-07,172.890,172.17,94554334.0,0.542038,-0.720,False
...,...,...,...,...,...,...,...,...
595,595,2024-05-16,190.470,189.84,52845230.0,-0.424816,-0.630,True
596,596,2024-05-17,189.510,189.87,41287835.0,0.154493,0.360,True
597,597,2024-05-20,189.325,191.04,44361275.0,0.433162,1.715,True
598,598,2024-05-21,191.090,192.35,42309401.0,0.084316,1.260,True


In [129]:
aapl_sent['difference'] = aapl_sent['close'] - aapl_sent['open']

In [135]:
aapl_sent['sign_match'] = np.sign(aapl_sent['difference']) == np.sign(aapl_sent['sent'])

In [137]:
aapl_sent['sign_match'].mean() * 100

51.66666666666667